# QGIS Development Environment Setup Using Docker

This notebook guides you through setting up a QGIS development environment using Docker. We'll cover cloning the repository, creating a Docker environment, and building QGIS from source.

## Prerequisites
- Docker installed on your system
- Docker Compose installed
- Git installed
- X11 server running (for Linux GUI applications)

## 1. Clone QGIS Repository

First, we'll clone the QGIS repository from GitHub. This contains all the source code needed to build QGIS.

In [ ]:
!git clone https://github.com/qgis/QGIS.git
%cd QGIS

## 2. Create Docker Compose Configuration

We'll create a `docker-compose.yml` file that defines our development environment. This configuration:
- Uses the official QGIS build dependencies image
- Mounts the source code and build directories
- Sets up X11 forwarding for GUI applications
- Configures networking for X11 support

In [ ]:
%%writefile docker-compose.yml
version: '3'

services:
  qgis-dev:
    image: qgis/qgis3-build-deps-22.04-qt5:latest
    volumes:
      - ..:/src/QGIS
      - build:/src/QGIS/build
    environment:
      - DISPLAY=${DISPLAY}
      - XAUTHORITY=${XAUTHORITY}
      - XDG_RUNTIME_DIR=/tmp
    network_mode: host
    working_dir: /src/QGIS/build
    command: bash

volumes:
  build:

## 3. Build the Development Environment

Now we'll build the Docker environment using the compose file we created.

In [ ]:
!docker compose build qgis-dev

## 4. Start the Development Environment

Launch the development container with an interactive bash shell.

In [ ]:
!docker compose run --rm qgis-dev bash

## 5. Building QGIS

Inside the container, we'll configure and build QGIS using CMake and Ninja. The configuration options control various features:

- `WITH_STAGED_PLUGINS`: Enable staged plugins
- `CMAKE_INSTALL_PREFIX`: Set installation directory for QGIS
- `WITH_GRASS`: Enable GRASS QGIS integration
- `SUPPRESS_QT_WARNINGS`: Reduce Qt warning messages
- `ENABLE_TESTS`: Disable testing for faster builds
- `WITH_QSPATIALITE`: Enable Spatialite support
- `WITH_DESKTOP`: Build desktop application
- `WITH_BINDINGS`: Enable Python bindings
- `DISABLE_DEPRECATED`: Disable deprecated functionality

In [ ]:
%%bash
# Configure with CMake
cmake -GNinja \
    -DWITH_STAGED_PLUGINS=ON \
    -DCMAKE_INSTALL_PREFIX=/usr \
    -DWITH_GRASS=ON \
    -DSUPPRESS_QT_WARNINGS=ON \
    -DENABLE_TESTS=OFF \
    -DWITH_QSPATIALITE=ON \
    -DWITH_QWTPOLAR=OFF \
    -DWITH_APIDOC=OFF \
    -DWITH_ASTYLE=OFF \
    -DWITH_DESKTOP=ON \
    -DWITH_BINDINGS=ON \
    -DDISABLE_DEPRECATED=ON

# Build QGIS
ninja

# Install QGIS
ninja install

## Notes and Troubleshooting

1. The build process may take significant time depending on your system resources.
2. Make sure your X11 server is properly configured for GUI applications.
3. If you encounter permission issues, you may need to run Docker commands with sudo.
4. The build volume persists between container runs, saving compilation time.

To run QGIS after installation, use the `qgis` command inside the container.